# Model 1: Fully Connected Neural Network

In [ ]:
import pandas as pd
import numpy as np
import torch 

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

In [ ]:
# Import local modules from 'src/utils' as package 'utils'
import sys; sys.path.insert(0, '../')

## Read Datasets from .csv

In [ ]:
from pathlib import Path
from utils.file_io import read_angle_datasets

In [ ]:
data_folder = Path("../../data/")
train_data, test_data = read_angle_datasets(data_folder, 0.9)
input_shape = train_data[0][0].shape[0]
print(input_shape)

## Model Definition

In [ ]:
from torch import nn
from typing import Tuple, List

In [ ]:
class FullyConnected(nn.Module):
    def __init__(self, flattened_input_dim: int, intermediate_dims: List, output_dim: int, dropout: float = 0.25, hidden_activation = nn.ReLU):
        super().__init__()
        self.total_epochs = 0
        self.flatten = nn.Flatten()
        self.hidden = nn.Sequential()
        for i, dim in enumerate(intermediate_dims):
            if i == 0:
                self.hidden.add_module(f"linear_{i+1}", nn.Linear(flattened_input_dim, dim))
            else:
                self.hidden.add_module(f"linear_{i+1}", nn.Linear(intermediate_dims[i-1], dim))

            self.hidden.add_module(f"hidden_activation_{i+1}", hidden_activation())
            self.hidden.add_module(f"dropout_{i+2}", nn.Dropout(dropout))

        self.last = nn.Linear(intermediate_dims[-1], output_dim)

    def forward(self, x):
        x = self.flatten(x)
        x = self.hidden(x)
        return self.last(x)
        

In [ ]:
model = FullyConnected(input_shape, [32, 16], 3).to(device)

## Training the model

In [ ]:
import time
import os
from dotenv import load_dotenv
from torch.utils.data import DataLoader, Subset
from utils.file_io import save_model
from utils.evaluation import compute_loss_on

In [ ]:
dotenv_path = Path("../../models/fully_connected/.env")
load_dotenv(dotenv_path=dotenv_path)
learning_rate = float(os.getenv("LEARNING_RATE"))
batch_size = int(os.getenv("BATCH_SIZE"))

# inputs for training loop
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_function = torch.nn.MSELoss()

train_size = len(train_data) - int(0.05 * len(train_data)) 
train_set = Subset(train_data, range(train_size))
validation_set = Subset(train_data, range(train_size, len(train_data)))
train_dataloader = DataLoader(train_set, batch_size=batch_size)
validation_dataloader = DataLoader(validation_set, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
def train(epochs: int, train_dataloader: DataLoader, validation_dataloader: DataLoader, model, loss_function, optimizer, checkpoint_path: Path, report_interval: int = 1000):
    best_val_loss = float("inf")
    for epoch in range(model.total_epochs, epochs):
        print(f"Epoch: {epoch + 1}")

        model.train(True)
        avg_loss = train_epoch(train_dataloader, model, loss_function, optimizer, report_interval)
        model.eval()

        with torch.no_grad():
            avg_val_loss = compute_loss_on(validation_dataloader, model, loss_function)

        print(f"Loss on train: {avg_loss}, loss on validation: {avg_val_loss}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            model_path = checkpoint_path / f"{checkpoint_path.name}_{epoch}.model"
            save_model(model, model_path)

        model.total_epochs += 1
    
    return model            


def train_epoch(train_dataloader: DataLoader, model, loss_function, optimizer, report_interval: int = 1000):
    running_loss = 0
    last_loss = 0
    
    for i, data in enumerate(train_dataloader):
        inputs, true_values = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, true_values)
        running_loss += loss
        loss.backward()
        optimizer.step()
    
    if i % report_interval == report_interval - 1:
        last_loss = running_loss / report_interval
        print(f"batch {i + 1}, Mean Squared Error: {last_loss}")
        running_loss = 0
    
    return last_loss


In [ ]:
checkpointing_path = Path("../../models/fully_connected/")
last_model = train(int(os.getenv("NUM_EPOCHS")), train_dataloader, validation_dataloader, model, loss_function, optimizer, checkpointing_path)

## Evaluation
### Compute mean squared error

In [ ]:
from utils.evaluation import compute_predictions, compute_losses_from

In [ ]:
y, y_true = compute_predictions(test_dataloader, last_model)
test_losses = compute_losses_from(y, y_true, loss_function)
print(f"The mean squared error on test is: {test_losses.mean()}")

### Draw prediction/truth traces 

In [ ]:
%matplotlib notebook

from matplotlib import pyplot as plt

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  

from IPython.display import HTML

from utils.visualization import create_trace_animation

In [ ]:
animation = create_trace_animation(y.numpy(), y_true.numpy())
HTML(animation.to_jshtml())

## Loading the best model

In [ ]:
from utils.file_io import load_model

In [ ]:
loaded_model = FullyConnected(input_shape, [32, 16], 3).to(device)
model_state_dict = load_model(checkpointing_path)
loaded_model.load_state_dict(model_state_dict)
loaded_model.eval()
y, y_true = compute_predictions(test_dataloader, loaded_model)
test_losses = compute_losses_from(y, y_true, loss_function)
print(f"The mean squared error of the loaded model on test is: {test_losses.mean()}")
animation = create_trace_animation(y.numpy(), y_true.numpy())
HTML(animation.to_jshtml())